importacao de biblioticas


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import os

Importacao e limpeza de dados


In [ ]:

pla_fib_1 = pd.read_csv('PLA FIBRA DE CARBONO/PLA FIBRA DE CARBONO_1.csv', encoding='ISO-8859-1', delimiter=';')    
pla_fib_2 = pd.read_csv('PLA FIBRA DE CARBONO/PLA FIBRA DE CARBONO_2.csv', encoding='ISO-8859-1', delimiter=';') 
pla_fib_3 = pd.read_csv('PLA FIBRA DE CARBONO/PLA FIBRA DE CARBONO_3.csv', encoding='ISO-8859-1', delimiter=';') 
pla_fib_4 = pd.read_csv('PLA FIBRA DE CARBONO/PLA FIBRA DE CARBONO_4.csv', encoding='ISO-8859-1', delimiter=';') 

pla_norm_1 = pd.read_csv('PLA NORMAL/PLA NORMAL_1_1.csv', encoding='utf-8', delimiter=';', skip_blank_lines=True)    
pla_norm_2 = pd.read_csv('PLA NORMAL/PLA NORMAL_2_1.csv', encoding='utf-8', delimiter=';', skip_blank_lines=True) 
pla_norm_3 = pd.read_csv('PLA NORMAL/PLA NORMAL_3_1.csv', encoding='utf-8', delimiter=';', skip_blank_lines=True) 
pla_norm_4 = pd.read_csv('PLA NORMAL/PLA NORMAL_4_1.csv', encoding='utf-8', delimiter=';', skip_blank_lines=True) 

pla_fibs = [pla_fib_1, pla_fib_2, pla_fib_3, pla_fib_4]
pla_norms = [pla_norm_1, pla_norm_2, pla_norm_3, pla_norm_4]


for i in range(len(pla_fibs)):
    pla_fibs[i] = pla_fibs[i].drop(0)
    pla_fibs[i] = pla_fibs[i].map(lambda x: x.replace(',', '.').replace('"', '') if isinstance(x, str) else x)
    pla_fibs[i] = pla_fibs[i].astype(float)

for i in range(len(pla_norms)):
    pla_norms[i] = pla_norms[i].drop(0)
    pla_norms[i] = pla_norms[i].map(lambda x: x.replace(',', '.').replace('"', '') if isinstance(x, str) else x)
    pla_norms[i] = pla_norms[i].astype(float)





Funcao de Calcular E (tks John)

In [ ]:
def calc_modulo_elasticidade_dinamico(df, limite_erro=0.01): #como entrada um DataFrame df e um limite_erro, que determina quão próximo da linearidade os dados devem estar.
    # Filtra valores de tensão positivos
    filtro = df

    # Inicializa variáveis
    n_max = len(filtro)
    modulo_elasticidade = np.nan

    for n in range(2, n_max):  # Começar com 2 pontos
        tensao = filtro['Tensão à tração'].iloc[:n].values      #Para cada iteração do loop, os dados de tensão e deformação são extraídos até o índice n. A deformação é convertida de porcentagem para fração dividindo por 100.
        deformacao = filtro['Deformação à tração (Deslocamento)'].iloc[:n].values / 100

        # Verifica se todos os valores de deformação ou tensão são iguais
        if np.all(deformacao == deformacao[0]) or np.all(tensao == tensao[0]):
            continue  # Ignora este conjunto e passa para o próximo

        # Realiza a regressão linear
        slope, intercept, r_value, p_value, std_err = linregress(deformacao, tensao) #Realiza a regressão linear nos dados de tensão e deformação. 
                                                                                           #slope representa o coeficiente angular da linha de melhor ajuste, 
                                                                                           # que é o que queremos calcular como o módulo de elasticidade.

        # Prediz a tensão usando a linha de melhor ajuste
        tensao_predita = slope * deformacao + intercept

        # Prediz os valores de tensão com base na linha de melhor ajuste e calcula o Erro Quadrático Médio (EQM) entre os valores reais e os preditos.
        eqm = np.mean((tensao - tensao_predita) ** 2)


#Se o EQM for menor que o limite definido, atualiza modulo_elasticidade com o valor do coeficiente angular (slope).
        # Verifica se o EQM é menor que o limite
        if eqm < limite_erro:
            modulo_elasticidade = slope  # O coeficiente angular é o módulo de elasticidade
        else:
            break  # Sai do loop se a linearidade não se mantiver

    return modulo_elasticidade


Geracao de dados importantes


In [17]:
print(f'FIBRA DE CARBONO')
for i in range(len(pla_fibs)):
    tensao = pla_fibs[i]['Tensão à tração']
    deformacao = pla_fibs[i]['Deformação à tração (Deslocamento)']

    modulo_elasticidade = calc_modulo_elasticidade_dinamico(pla_fibs[i])


    # Filter to get the plastic region (from 0.02 strain to the end)
    plastic_region = pla_fibs[i][(pla_fibs[i]['Deformação à tração (Deslocamento)'] >= 1)]
    tensao_plastic = plastic_region['Tensão à tração']
    deformacao_plastic = plastic_region['Deformação à tração (Deslocamento)']

    # Calculate the plastic modulus using linear regression
    slope_plastic, intercept_plastic, _, _, _ = linregress(deformacao_plastic/100, tensao_plastic)
    modulo_plastico = slope_plastic

    resistencia_maxima = tensao.max()
    deformacao_maxima = deformacao[tensao.idxmax()] #deformacao na resistencia maxima
    resistencia_ruptura = tensao.iloc[-1]
    deformacao_ruptura = deformacao.iloc[-1]
    tenacidade = np.trapezoid(tensao, deformacao)
    poisson = np.abs(modulo_plastico/modulo_elasticidade) * 10


    print(f"Amostra {i+1}:")
    print(f" - Módulo de Elasticidade: {modulo_elasticidade:.2f} MPa")
    print(f" - Resistência Máxima: {resistencia_maxima:.2f} MPa")
    print(f" - Deformação na Resistência Máxima: {deformacao_maxima:.4f}%")
    print(f" - Resistência na Ruptura: {resistencia_ruptura:.2f} MPa")
    print(f" - Deformação na Ruptura: {deformacao_ruptura:.4f}")
    print(f" - Tenacidade: {tenacidade:.2f} J/m³")
    print(f" - Coeficiente de Poisson: {poisson:.2f}")
    print("-" * 40)


print(f'PLA NORMAL')
for i in range(len(pla_norms)):
    tensao = pla_norms[i]['Tensão à tração']
    deformacao = pla_norms[i]['Deformação à tração (Deslocamento)']

    modulo_elasticidade = calc_modulo_elasticidade_dinamico(pla_norms[i])
    
    # Filter to get the plastic region (from 0.02 strain to the end)
    plastic_region = pla_norms[i][(pla_norms[i]['Deformação à tração (Deslocamento)'] >= 1.5) ]
    tensao_plastic = plastic_region['Tensão à tração']
    deformacao_plastic = plastic_region['Deformação à tração (Deslocamento)']

    # Calculate the plastic modulus using linear regression
    slope_plastic, intercept_plastic, _, _, _ = linregress(deformacao_plastic/100, tensao_plastic)
    modulo_plastico = slope_plastic

    resistencia_maxima = tensao.max()
    deformacao_maxima = deformacao[tensao.idxmax()] #deformacao na resistencia maxima
    resistencia_ruptura = tensao.iloc[-1]
    deformacao_ruptura = deformacao.iloc[-1]
    tenacidade = np.trapezoid(tensao, deformacao)
    poisson = np.abs(modulo_plastico/modulo_elasticidade)/10


    print(f"Amostra {i+1}:")
    print(f" - Módulo de Elasticidade: {modulo_elasticidade:.2f} MPa")
    print(f" - Resistência Máxima: {resistencia_maxima:.2f} MPa")
    print(f" - Deformação na Resistência Máxima: {deformacao_maxima:.4f}%")
    print(f" - Resistência na Ruptura: {resistencia_ruptura:.2f} MPa")
    print(f" - Deformação na Ruptura: {deformacao_ruptura:.4f}")
    print(f" - Tenacidade: {tenacidade:.2f} J/m³")
    print(f" - Coeficiente de Poisson: {poisson:.2f}")
    print("-" * 40)


FIBRA DE CARBONO
Amostra 1:
 - Módulo de Elasticidade: 2608.57 MPa
 - Resistência Máxima: 21.01 MPa
 - Deformação na Resistência Máxima: 1.3755%
 - Resistência na Ruptura: 10.02 MPa
 - Deformação na Ruptura: 6.9259
 - Tenacidade: 120.58 J/m³
 - Coeficiente de Poisson: 0.24
----------------------------------------
Amostra 2:
 - Módulo de Elasticidade: 2715.27 MPa
 - Resistência Máxima: 23.30 MPa
 - Deformação na Resistência Máxima: 1.5734%
 - Resistência na Ruptura: 11.91 MPa
 - Deformação na Ruptura: 5.3663
 - Tenacidade: 104.88 J/m³
 - Coeficiente de Poisson: 0.46
----------------------------------------
Amostra 3:
 - Módulo de Elasticidade: 2719.57 MPa
 - Resistência Máxima: 20.69 MPa
 - Deformação na Resistência Máxima: 1.3492%
 - Resistência na Ruptura: 9.88 MPa
 - Deformação na Ruptura: 6.4485
 - Tenacidade: 110.61 J/m³
 - Coeficiente de Poisson: 0.26
----------------------------------------
Amostra 4:
 - Módulo de Elasticidade: 2726.74 MPa
 - Resistência Máxima: 24.35 MPa
 - Defo

Plotagem dos graficos


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# First figure for 'pla_fibs' data
plt.figure(figsize=(8, 6))
for i in range(len(pla_fibs)):
    deslocamento = pla_fibs[i]['Deformação à tração (Deslocamento)']
    tensao_flexao = pla_fibs[i]['Tensão à tração']

    # Filter data from 0 to 2.5
    '''
    mask = (deslocamento >= 0) & (deslocamento <= 2.5)
    deslocamento = deslocamento[mask]
    tensao_flexao = tensao_flexao[mask]
    '''
    
    # Plot main data
    plt.plot(deslocamento, tensao_flexao, label=f'Amostra: {i + 1}', color=colors[i])
    
    # Calculate standard deviation and fill
    std_dev = np.std(tensao_flexao)
    plt.fill_between(deslocamento, tensao_flexao - std_dev, tensao_flexao + std_dev,
                     color=colors[i], alpha=0.2, label=f'Std. Dev: {std_dev:.2f}')


    # Calculate tangent line at the middle point
    mid_idx = len(deslocamento) //12 # index for the middle point
    x0, y0 = deslocamento[mid_idx], tensao_flexao[mid_idx]
    slope, intercept = np.polyfit(deslocamento[mid_idx-1:mid_idx+2], tensao_flexao[mid_idx-1:mid_idx+2], 1)
    tangent_line = slope * (deslocamento - x0) + y0

  # Limit tangent line values between y = 40 and y = 80
    mask = (tangent_line >= 10) & (tangent_line <= 30)
    deslocamento = deslocamento[mask]
    tangent_line = tangent_line[mask]


    # Plot tangent line
    plt.plot(deslocamento, tangent_line, '--', color=colors[i])
    plt.xlim(0, 7)
    plt.ylim(0, 35)

    # Add labels, grid, title, and legend
    plt.xlabel('Deformação à tração (%)')
    plt.ylabel('Tensão à flexão (MPa)')
    plt.title(f'Tensão vs Deformação: PLA FIBRA DE CARBONO')
    plt.grid(True)
    plt.legend()
plt.savefig(os.path.join('figs', f'PLA_FIBRA_DE_CARBONO.png'))

# Second figure for 'pla_norms' data
plt.figure(figsize=(8, 6))
for i in range(len(pla_norms)):
    deslocamento = pla_norms[i]['Deformação à tração (Deslocamento)']
    tensao_flexao = pla_norms[i]['Tensão à tração']

    # Plot main data
    plt.plot(deslocamento, tensao_flexao, label=f'Amostra: {i + 1}', color=colors[i])
    
    # Calculate standard deviation and fill
    std_dev = np.std(tensao_flexao)
    plt.fill_between(deslocamento, tensao_flexao - std_dev, tensao_flexao + std_dev,
                     color=colors[i], alpha=0.2, label=f'Std. Dev: {std_dev:.2f}')

    # Calculate tangent line at the middle point
    mid_idx = len(deslocamento) //2
    x0, y0 = deslocamento[mid_idx], tensao_flexao[mid_idx]
    # Use numpy's polyfit to get the slope of the tangent line at the middle point
    slope, intercept = np.polyfit(deslocamento[mid_idx-1:mid_idx+2], tensao_flexao[mid_idx-1:mid_idx+2], 1)
    tangent_line = slope * (deslocamento - x0) + y0


    # Limit tangent line values between y = 40 and y = 80
    mask = (tangent_line >= 10) & (tangent_line <= 30)
    deslocamento = deslocamento[mask]
    tangent_line = tangent_line[mask]

    # Plot tangent line
    plt.plot(deslocamento, tangent_line, '--', color=colors[i])
    plt.xlim(0, 2)
    plt.ylim(0, 50)

    # Add labels, grid, title, and legend
    plt.xlabel('Deformação à tração (%)')
    plt.ylabel('Tensão à tração (MPa)')
    plt.title(f'Tensão vs Deformação: PLA_NORMAL')
    plt.grid(True)
    plt.legend()
plt.savefig(os.path.join('figs', f'PLA NORMAL'))
